In [1]:
import cv2
import numpy as np
from cvlib import detect_face
from keras.models import load_model
from PIL.Image import open as Image_open
model = load_model("best_model.h5") # Loading the pretrained model to predict

Take sample images from Google for each class and name them as the ones (i.e <code>"sad.jpg"</code>) below. Now, let's find out the class numbers that the model will assign because sometimes we end up arbitrary numbers instead of our class numbers.

In [2]:
# Get image > Convert to numpy array > Resize it to our model's input_shape:
happy_img = cv2.resize((np.array(Image_open("happy.jpg"))), (64,64))
sad_img = cv2.resize((np.array(Image_open("sad.jpg"))), (64,64))
shocked_img = cv2.resize((np.array(Image_open("shocked.jpg"))), (64,64))
neutral_img = cv2.resize((np.array(Image_open("neutral.jpg"))), (64,64))
# Convert to grayscale > Normalize them:
happy_img = cv2.cvtColor(happy_img, code=6) / 255.0
sad_img = cv2.cvtColor(sad_img, code=6) / 255.0
shocked_img = cv2.cvtColor(shocked_img, code=6) / 255.0
neutral_img = cv2.cvtColor(neutral_img, code=6) / 255.0
# Add dimensions to match the model shape
# i.e (IMG_COUNT, IMG_DIMENSION, IMG_DIMENSION, COLOR_CHANNEL)
# (1, 64, 64, 1)
happy_img = np.expand_dims(happy_img, axis=[0,3])
sad_img = np.expand_dims(sad_img, axis=[0,3])
shocked_img = np.expand_dims(shocked_img, axis=[0,3])
neutral_img = np.expand_dims(neutral_img, axis=[0,3])

print("happy:", model.predict(happy_img))
print("sad:", model.predict(sad_img))
print("shocked:", model.predict(shocked_img))
print("poker face:", model.predict(neutral_img))

happy: [[1.2646025e-04 1.6763769e-03 9.1766296e-03 9.8902053e-01]]
sad: [[0.04921061 0.7129909  0.20824736 0.02955114]]
shocked: [[9.9780884e-08 1.1519273e-05 9.9998438e-01 4.0535142e-06]]
poker face: [[0.12971415 0.35642865 0.47015446 0.04370266]]


In [3]:
def getClassName(classIndex):
    "A basic function to get the class name."
    if classIndex == 3: return "Happy"
    elif classIndex == 1: return "Sad"
    elif classIndex == 2: return "Shocked"
    else: return "Poker Face"

In [5]:
cap = cv2.VideoCapture(0)
# If a video name is entered, captures the specified video.

detectFace_threshold = 0.80
predictFace_threshold = 0.39 * 100

try:
    while cap.isOpened(): # Repeats until a key is pressed.

        ret, frame = cap.read() # Starts capturing. 
        # 'ret' is True or False to represent if capturing is successful.
        # 'frame' is the image to process.
        if not ret:
            print("Can't receive the frame.")
            break

        # FACE RECOGNITION
        faces, confidences = detect_face(frame, threshold=detectFace_threshold)
        for f in faces:
            # corner points of facial frame: 
            (startX, startY) = (f[0], f[1])    # top left corner
            (endX, endY) = (f[2], f[3])    # bottom right corner
            # crop it from the whole frame:
            cropped_frame = np.copy(frame[startY:endY, startX:endX])        
            # skip too small frames (10x10 pixels)
            if (cropped_frame.shape[0]) < 10 or (cropped_frame.shape[1]) < 10:
                continue

            # preprocessing on the cropped frame
            cropped_frame = cv2.resize(cropped_frame, (64,64))        
            cropped_frame = cv2.cvtColor(cropped_frame, code=6)    # convert to grayscale
            cropped_frame = cropped_frame.astype("float32") / 255.0
            cropped_frame = np.expand_dims(cropped_frame, axis=[0,3])
            confidences = model.predict(cropped_frame)[0]    # probability value
            max_probability = max(confidences)*100
            classIndex = np.argmax(confidences)
            className = getClassName(classIndex)

            # different colors for different sentiments
            if max_probability > predictFace_threshold:
                frame_text = f"{className} ({int(max_probability)}%)"
                # BLUE - GREEN - RED rgb(218,112,214)
                if className == "Shocked":
                    rect_color = text_color = (0, 255, 255)
                elif className == "Sad":
                    rect_color = text_color = (255, 0, 0)
                elif className == "Happy":
                    rect_color = text_color = (0, 255, 0)
                elif className == "Poker Face":
                    rect_color = text_color = (214, 112, 218)

            else:
                rect_color = text_color = (0, 0, 255)
                frame_text = "Reading..."    

            cv2.rectangle(img=frame, 
                          pt1=(startX, startY), 
                          pt2=(endX, endY), 
                          color=rect_color, 
                          thickness=2)

            # let's keep the text in the frame and avoid edges:
            startY = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.putText(img=frame, 
                        text=frame_text, 
                        org=(startX, startY), 
                        fontFace=cv2.FONT_HERSHEY_COMPLEX, 
                        lineType=cv2.LINE_AA,
                        fontScale=0.6, 
                        color=text_color, 
                        thickness=1)

        cv2.imshow("Face_Sentiment", frame)
        cv2.imwrite("saved_picture.jpg", frame)
        if cv2.waitKey(1) > 0:    # Repeats until a key is pressed.
            break

    cap.release() # Releases the camera after a key is pressed
    cv2.destroyWindow("Face_Sentiment") # Closes the window named "Face_Sentiment"
    # cv2.waitKey() # Necessary for not having trouble with the closing window
except:
    cap.release()
    cv2.destroyAllWindows()